<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Параметры-для-запуска" data-toc-modified-id="Параметры-для-запуска-1">Параметры для запуска</a></span></li><li><span><a href="#Чтение-данных" data-toc-modified-id="Чтение-данных-2">Чтение данных</a></span></li></ul></div>

In [1]:
import os
import sys

In [2]:
from tqdm.auto import tqdm

In [3]:
import napari

In [4]:
import scipy
from scipy import ndimage as ndi
import numpy as np
import networkx as nx

from pathlib import Path

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import pickle
from astrocyte import Astrocyte

In [7]:
import ccdb
import astromorpho as astro

Can't load imreg package, affine and homography registrations won't work


/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/imfun/external/tifffile.py:7261: UserWarning: No module named 'imfun.external._tifffile'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/imfun/external/czifile.py:146: UserWarning: ImportError: No module named '_czifile'. Decoding of JXR and JPEG encoded images will not be available. Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  warnings.warn(
2022-09-06 14:23:45.692944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/levtg/astro-morpho/venv/lib/python3.10/site-packages/cv2/../../lib64:/home/levtg/v3d_external/v3d_main/common_lib/lib/
2022-09-06 14:23:45.692967: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dler

In [8]:
import graph_utils as gu  
import visualization as vis

In [9]:
import networkx as nx

In [10]:
from networx2napari import draw_edges, draw_nodes

In [11]:
import astro_graph

# Параметры для запуска

In [12]:
if os.path.exists('/home/levtg/astro-morpho'):
#     data_dir = '/home/levtg/astro-morpho/data/graphs'
    data_dir = '/home/levtg/astro-morpho/src/output/'
else:
    print("Dont know where to look for the data")

output_dir = './output/'

filename = '4wk-ly9-raw.pic-sequential_graph.pickle' # Octopus

VERBOSE = True

# Set false to start from console
HANDY = True

# Set true to save output
OUT = False

# Чтение данных

In [13]:
filepath = Path(data_dir).joinpath(filename)
filepath

PosixPath('/home/levtg/astro-morpho/src/output/4wk-ly9-raw.pic-sequential_graph.pickle')

In [154]:
%time data = pickle.load(open(filepath, 'rb'))

CPU times: user 40.7 ms, sys: 35.7 ms, total: 76.4 ms
Wall time: 76 ms


In [155]:
w =  napari.view_image(data.image, ndisplay=3, opacity=0.5)
vis.view_graph_as_colored_image(data.graph, data.parameters['shape'], w);

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [156]:
roots = gu.get_roots(data.graph)
roots

{(18, 153, 172),
 (19, 159, 170),
 (19, 159, 171),
 (20, 132, 173),
 (20, 159, 169),
 (20, 160, 170),
 (21, 144, 170),
 (21, 156, 167),
 (21, 158, 169),
 (21, 159, 170),
 (22, 130, 176),
 (22, 145, 170),
 (22, 148, 167),
 (22, 149, 166),
 (22, 167, 195),
 (23, 135, 194),
 (23, 147, 169),
 (23, 151, 167),
 (27, 170, 190),
 (27, 173, 187),
 (28, 170, 189),
 (29, 161, 183),
 (29, 170, 182),
 (30, 166, 181)}

In [157]:
branches = {}
for root in roots:
    branches[root] = astro_graph.AstroGraph(gu.filter_graph(data.graph, lambda node: node['root'] == root))

Смотрим на веточки

In [158]:
import itertools as itt

w = napari.view_image(data.image, opacity=0.5)
pos = {node: node for node in data.graph.nodes}

colors = ['blue', 'red', 'yellow', 'cyan', 'green', 'magenta', 'bop orange']
for branch, color in zip(branches.values(), itt.cycle(colors)):
    img = np.zeros(data.image.shape)
    points = draw_nodes(pos, branch.nodes())
    img[points[:, 0], points[:, 1], points[:, 2]] = 1
    w.add_image(img, colormap=color, blending='additive')
napari.run()

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [159]:
roots_arr = np.array(list(roots))

Объединяем ветки в пучки основываясь на расстоянии между корневыми точками

In [160]:
bunches = []
min_dist = 4

for root in roots:
    roots_dists = np.linalg.norm(root - roots_arr, axis=-1)
#     for r, d in zip(roots_arr[roots_dists < min_dist], roots_dists[roots_dists < min_dist]):
#         print(root , r, d)
    neighbours = [tuple(r) for r in roots_arr[roots_dists < min_dist]]
    STOP=False
    for bunch in bunches:
        for n in neighbours:
            if n in bunch:
                bunch.update(neighbours)
                STOP = True
                continue
        
        if STOP:
            break
    if not STOP:
        bunches.append(set(neighbours))

Часть пучков содержит одни и те же ветки. Объеденим такие пучки в один, а лишние удалим

In [161]:
set2del = []

for i, cur_bunch in enumerate(bunches[:-1]):
    FOUND = False
    for node in cur_bunch:
        for bunch in bunches[i+1:]:
            if node in bunch:
                bunch.update(cur_bunch)
                set2del.append(i)
                FOUND = True
                break
        if FOUND:
            break


In [162]:
bunches.pop(*set2del)

{(21, 144, 170),
 (22, 145, 170),
 (22, 148, 167),
 (22, 149, 166),
 (23, 147, 169),
 (23, 151, 167)}

Смотрим на результат

In [163]:
w = napari.view_image(data.image, opacity=0.5)
pos = {node: node for node in data.graph.nodes}
colors = ['blue', 'red', 'yellow', 'cyan', 'green', 'magenta', 'bop orange']


for bunch, color in zip(bunches, itt.cycle(colors)):
    img = np.zeros(data.image.shape)
    for root in bunch:
        points = draw_nodes(pos, branches[root].nodes())
        img[points[:, 0], points[:, 1], points[:, 2]] = 1
    w.add_image(img, colormap=color, blending='additive')

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [164]:
def choose_main(bunch, branches, mass_func=len):
    max_mass = 0
    for branch_root in bunch:
        branch = branches[branch_root]
        branch_mass = mass_func(branch)
        if branch_mass > max_mass:
            max_mass = branch_mass
            main_branch_root = branch_root
            main_branch = branches[main_branch_root]
    return main_branch_root, main_branch

Выбираем первый (ближайший к корню) участок ветки и рассматриваем его положение относительно основной ветки. 

In [ ]:
# w = napari.view_image(data.image, opacity=0.5)
# pos = {node: node for node in data.graph.nodes}

# for branch, color in zip(bunch, itt.cycle(colors)):
#     img = np.zeros(data.image.shape)
#     astroGraph = astro_graph.AstroGraph(branches[branch])
#     path = list(astroGraph.graph_to_paths().values())[0][0]
#     points = draw_nodes(pos, path)
    
#     img[points[:, 0], points[:, 1], points[:, 2]] = 1
#     w.add_image(img, colormap=color, blending='additive')    

In [ ]:
# # Adding main branch
# img = np.zeros(data.image.shape)
# astroGraph = astro_graph.AstroGraph(branches[main_branch_root])
# path = list(astroGraph.graph_to_paths().values())[0][0]
# points = draw_nodes(pos, path)

# img[points[:, 0], points[:, 1], points[:, 2]] = 1
# w.add_image(img, colormap='gray', blending='additive', name='main branch') 

Выбираем первый (ближайший к корню) участок ветки и рассматриваем его положение относительно основной ветки. Если расстояние между ветками от корневых точек не увеличивается, то точки расссматриваемой ветки удалаются, а последующая соединяется с соответствующей точкой на основной ветке

In [172]:
# def draw_points(viewer, points, img_shape, color='red'):
#     img = np.zeros(img_shape)
#     img[points[:, 0], points[:, 1], points[:, 2]] = 1
#     viewer.add_image(img, colormap=color, blending='additive')

In [165]:
def make_lines(branch, root):
    lines = {}
    for tip in branch.get_tips():
        lines[tip] = nx.shortest_path(branch.graph, root, tip), nx.shortest_path_length(branch.graph, root, tip)
    return lines

In [ ]:
max_dist = 4

lines = make_lines(branch, branch_root)
main_line = choose_main(branch.get_tips(), lines)
for line in lines:
    parallel = []
    cur_dist = 0
    start_dist = 0
    for l, ml in zip(line, main_line):
        if 

In [166]:
for bunch in bunches:
    main_branch_root, main_branch = choose_main(bunch, branches, lambda x: len(x.nodes()))
    main_branch_lines = make_lines(main_branch, main_branch_root)
    main_branch_line_tip, (main_branch_line, main_branch_line_mass) = choose_main(main_branch.get_tips(), main_branch_lines)
    main_branch_points = draw_nodes(pos, main_branch_line)

    for branch_root in tqdm(bunch):
        if branch_root == main_branch_root:
            continue
        branch = branches[branch_root]
        nx.set_node_attributes(data.graph, {p: main_branch_root for p in branch.nodes()}, name='root')
        

    
        for line, line_mass in make_lines(branch, branch_root).values():
            points = draw_nodes(pos, line)

    #         branch_paths = list(branch.graph_to_paths().values())
    #         for path in branch_paths[0]:
    #             path = [branch_root] + path
    #             points = draw_nodes(pos, path)

            count = min(len(points), len(main_branch_points))
            dists = np.linalg.norm(points[:count] - main_branch_points[:count], axis=-1)
            REMOVED = False
            for p, mbp, d in zip(points[:count], main_branch_points[:count], dists):
                point = p
                mb_point = mbp
#                 print(d, data.graph.nodes[tuple(mbp)]['sigma_mask'])
                
                if tuple(p) not in data.graph:
                    continue
                elif data.graph.nodes[tuple(p)]['sigma_mask'] == data.graph.nodes[tuple(mbp)]['sigma_mask'] \
                    or d <= min_dist:
#                 min(data.graph.nodes[tuple(mbp)]['sigma_opt'], data.graph.nodes[tuple(p)]['sigma_opt']):
                    data.graph.remove_node(tuple(p))
                    REMOVED = True
                else:
#                     print('Mask: {:0.2f} {:0.2f} {:0.3f}'.format(data.graph.nodes[tuple(mbp)]['sigma_mask'], data.graph.nodes[tuple(p)]['sigma_mask'], d, end=' '))
#                     print('Opt: {:0.2f} {:0.2f} {:0.3f}'.format(data.graph.nodes[tuple(mbp)]['sigma_opt'], data.graph.nodes[tuple(p)]['sigma_opt'], d))
                    break

            else:
                point = mb_point

            if REMOVED:
                cur_p = mb_point 
                prev_p = mb_point
                end_p = point
                azi = np.array([*np.sign(end_p - cur_p)])

                while tuple(cur_p) != tuple(end_p):
                    cur_p = np.clip(cur_p + azi, np.min([point, mb_point], axis=0), np.max([point, mb_point], axis=0))
                    data.graph.add_node(tuple(cur_p), root=main_branch_root,
                                                      sigma_mask=data.parameters['id2sigma'][data.sigma_mask[cur_p[0], cur_p[1], cur_p[2]]]) #Add another parameters
                    data.graph.add_edge(tuple(prev_p), tuple(cur_p))
                    prev_p = cur_p

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [145]:
data.sigma_mask.shape

(55, 464, 312)

In [168]:
pos_ = {node: node for node in data.graph.nodes}

w =  napari.view_image(data.image, ndisplay=3, opacity=0.5)
img = np.zeros(data.image.shape)

points = draw_nodes(pos_, data.graph.nodes())

img[points[:, 0], points[:, 1], points[:, 2]] = 1
w.add_image(img, colormap='gray', blending='additive', name='main branch')

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'main branch' at 0x7f60d5496fe0>

In [167]:
w =  napari.view_image(data.image, ndisplay=3, opacity=0.5)
vis.view_graph_as_colored_image(data.graph, data.parameters['shape'], w);

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


RecursionError: maximum recursion depth exceeded while calling a Python object

In [112]:
w.add_image(data.sigma_mask)

<Image layer 'Image [4]' at 0x7f6164262c50>

In [81]:
# np.clip(p+np.array([*np.sign(ep - p)]), np.min([p, ep], axis=0), np.max([p, ep], axis=0))

array([ 28, 131, 142])